Install dependencies

In [25]:
!pip install -U transformers accelerate evaluate peft einops bitsandbytes
!pip install datasets==2.15.0 
!pip install trl

  Obtaining dependency information for trl from https://files.pythonhosted.org/packages/0d/44/c406c3cf5981bddb16ff72acb5ca235888db4073d868cf51bd143bef3aad/trl-0.7.4-py3-none-any.whl.metadata
  Obtaining dependency information for tyro>=0.5.11 from https://files.pythonhosted.org/packages/c5/11/abdf67467d06713b431618732a43f82d1b1f02120107b05a789afbcdf54d/tyro-0.6.0-py3-none-any.whl.metadata
  Obtaining dependency information for shtab>=1.5.6 from https://files.pythonhosted.org/packages/40/ad/7227da64498eaa7abecee4311008f70869e156014b3270cec36e2e70cd31/shtab-1.6.5-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 1.3 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 5.3 MB/s eta 0:00:00


In [2]:
import torch
from huggingface_hub import notebook_login
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model


**Hugging face authentication**

In [11]:
notebook_login()

**Bits and Bytes encoding**

In [4]:
bb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [34]:
model_name = "microsoft/phi-1"
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto", device_map="cuda", trust_remote_code=True,  quantization_config=bb_config)
model.config.use_cache = False


config.json:   0%|          | 0.00/730 [00:00<?, ?B/s]

configuration_phi.py:   0%|          | 0.00/2.03k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-1:
- configuration_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi.py:   0%|          | 0.00/33.5k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-1:
- modeling_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/2.84G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/69.0 [00:00<?, ?B/s]

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [8]:
print(model)

PhiForCausalLM(
  (transformer): PhiModel(
    (embd): Embedding(
      (wte): Embedding(51200, 2560)
      (drop): Dropout(p=0.0, inplace=False)
    )
    (h): ModuleList(
      (0-31): 32 x ParallelBlock(
        (ln): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.1, inplace=False)
        (mixer): MHA(
          (rotary_emb): RotaryEmbedding()
          (Wqkv): Linear4bit(in_features=2560, out_features=7680, bias=True)
          (out_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (inner_attn): SelfAttention(
            (drop): Dropout(p=0.0, inplace=False)
          )
          (inner_cross_attn): CrossAttention(
            (drop): Dropout(p=0.0, inplace=False)
          )
        )
        (mlp): MLP(
          (fc1): Linear4bit(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear4bit(in_features=10240, out_features=2560, bias=True)
          (act): NewGELUActivation()
        )
      )

In [9]:
lora_config = LoraConfig(
    r=8, 
    lora_alpha=32, 
    target_modules=["Wqkv"], 
    lora_dropout=0.05, 
    bias="none", 
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
print_trainable_parameters(model)

trainable params: 2621440 || all params: 1524014080 || trainable%: 0.1720089095239855


**Load the open assistant dataset**

In [10]:
dataset = load_dataset("OpenAssistant/oasst1")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/84437 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4401 [00:00<?, ? examples/s]

In [12]:
df = dataset["train"].to_pandas()
df.info(verbose=True, memory_usage=True, show_counts=True)
df.sample(1).transpose().to_dict()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84437 entries, 0 to 84436
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   message_id       84437 non-null  object 
 1   parent_id        74591 non-null  object 
 2   user_id          84437 non-null  object 
 3   created_date     84437 non-null  object 
 4   text             84437 non-null  object 
 5   role             84437 non-null  object 
 6   lang             84437 non-null  object 
 7   review_count     84437 non-null  int32  
 8   review_result    83732 non-null  object 
 9   deleted          84437 non-null  bool   
 10  rank             48730 non-null  float64
 11  synthetic        84437 non-null  bool   
 12  model_name       0 non-null      object 
 13  detoxify         72297 non-null  object 
 14  message_tree_id  84437 non-null  object 
 15  tree_state       84437 non-null  object 
 16  emojis           71496 non-null  object 
 17  labels      

{69077: {'message_id': '6a529afd-151e-4532-a565-59c95ff2c0bb',
  'parent_id': 'abe94864-3205-46cd-9a4a-956fd46b89a4',
  'user_id': '95f63ab3-6c7b-43e1-8e86-fccd8be1465f',
  'created_date': '2023-02-04T16:05:16.054891+00:00',
  'text': "Sure! Here's an example of a simple project checklist for building a website:\n\nDefine the purpose and goals of the website\nResearch and choose a domain name\nPurchase the domain name and select a web host\nPlan the website structure and layout\nCreate and gather content (text, images, videos, etc.)\nChoose a website builder or design and develop the website using HTML, CSS, and JavaScript (or hire a professional)\nTest the website for compatibility and functionality\nLaunch the website and promote it through search engine optimization and marketing\nMonitor website traffic and analytics\nRegularly update and maintain the website\nThis is just an example, and you can adjust the items in this checklist based on your specific needs and goals. The importa

In [ ]:
train_dataset = dataset['train']
valid_dataset = dataset['validation']
#train_dataset_mapped = train_dataset.map(lambda examples: {'text': [f'[INST] <>\n{system_message.strip()}\n<>\n\n' + prompt + ' [/INST] ' + response for prompt, response in zip(examples['prompt'], examples['response'])]}, batched=True)
#valid_dataset_mapped = valid_dataset.map(lambda examples: {'text': [f'[INST] <>\n{system_message.strip()}\n<>\n\n' + prompt + ' [/INST] ' + response for prompt, response in zip(examples['prompt'], examples['response'])]}, batched=True)


In [22]:
# lets grab a random message tree
message_tree_id = 'd0bfc03e-9cf9-4465-bf99-53b90e071a8f'
print(message_tree_id)
msg_tree = df.query(f"message_tree_id == '{message_tree_id}'").sort_values("created_date")



d0bfc03e-9cf9-4465-bf99-53b90e071a8f


,message_id,parent_id,user_id,created_date,text,role,lang,review_count,review_result,deleted,rank,synthetic,model_name,detoxify,message_tree_id,tree_state,emojis,labels
69070,d0bfc03e-9cf9-4465-bf99-53b90e071a8f,None,2998ffeb-b867-49be-8f75-93ec86aa2f38,2023-01-31T04:49:16.777708+00:00,I want to do a project to label data for train...,prompter,en,3,True,False,NaN,False,None,"{'toxicity': 0.00034056275035254657, 'severe_t...",d0bfc03e-9cf9-4465-bf99-53b90e071a8f,ready_for_export,"{'name': ['+1'], 'count': [11]}","{'name': ['spam', 'lang_mismatch', 'pii', 'not..."
69071,c16d84a5-fb69-465e-83c5-7c046d1e4a65,d0bfc03e-9cf9-4465-bf99-53b90e071a8f,d152cc81-57da-4461-a213-48771c6dcd31,2023-02-02T07:35:34.672501+00:00,It can be challenging to focus when faced with...,assistant,en,3,True,False,0.0,False,None,"{'toxicity': 0.00028685241704806685, 'severe_t...",d0bfc03e-9cf9-4465-bf99-53b90e071a8f,ready_for_export,None,"{'name': ['spam', 'fails_task', 'lang_mismatch..."
69072,fe34dab0-bab3-4052-8e35-13db58521d9c,c16d84a5-fb69-465e-83c5-7c046d1e4a65,56916028-db43-4c8a-82c8-7a217c042cfe,2023-02-02T11:33:35.831727+00:00,Do you have any information on how the data wa...,prompter,en,3,True,True,NaN,False,None,"{'toxicity': 0.00083920918405056, 'severe_toxi...",d0bfc03e-9cf9-4465-bf99-53b90e071a8f,ready_for_export,"{'name': ['red_flag', '_skip_reply'], 'count':...","{'name': ['spam', 'lang_mismatch', 'pii', 'not..."
69079,95907393-b54b-4c74-b582-b9a4be408e84,d0bfc03e-9cf9-4465-bf99-53b90e071a8f,00bbfff8-0e6a-4e46-93cc-65785d78c8bc,2023-02-02T22:22:37.439964+00:00,It sounds like you are feeling overwhelmed by ...,assistant,en,3,True,False,2.0,False,None,"{'toxicity': 0.0003732253680936992, 'severe_to...",d0bfc03e-9cf9-4465-bf99-53b90e071a8f,ready_for_export,"{'name': ['+1', '_skip_reply'], 'count': [6, 2]}","{'name': ['spam', 'fails_task', 'lang_mismatch..."
69074,21d964a4-bb10-4cc0-8374-9b1d4d4f8662,d0bfc03e-9cf9-4465-bf99-53b90e071a8f,fa6353b9-0c47-4702-a14f-0d2031ddd42a,2023-02-03T06:15:29.855482+00:00,When you're feeling overwhelmed by a seemingly...,assistant,en,3,True,False,1.0,False,None,"{'toxicity': 0.00043202543747611344, 'severe_t...",d0bfc03e-9cf9-4465-bf99-53b90e071a8f,ready_for_export,"{'name': ['+1', '_skip_reply'], 'count': [9, 2]}","{'name': ['spam', 'fails_task', 'lang_mismatch..."
69075,abe94864-3205-46cd-9a4a-956fd46b89a4,21d964a4-bb10-4cc0-8374-9b1d4d4f8662,c50e38ea-e1c1-4e29-9b1b-10d113e6fcc5,2023-02-03T16:55:45.337378+00:00,Please provide me an example of such a checklist,prompter,en,3,True,False,NaN,False,None,"{'toxicity': 0.0004868093819823116, 'severe_to...",d0bfc03e-9cf9-4465-bf99-53b90e071a8f,ready_for_export,"{'name': ['+1', '_skip_reply', '_skip_ranking'...","{'name': ['spam', 'lang_mismatch', 'pii', 'not..."
69080,0e502c04-8ce5-4014-8b2e-85c856ecd267,95907393-b54b-4c74-b582-b9a4be408e84,fa6353b9-0c47-4702-a14f-0d2031ddd42a,2023-02-03T17:22:26.241609+00:00,I don't know where to start! What should I do ...,prompter,en,3,True,False,NaN,False,None,"{'toxicity': 0.00047203953727148473, 'severe_t...",d0bfc03e-9cf9-4465-bf99-53b90e071a8f,ready_for_export,"{'name': ['+1', '_skip_reply', '_skip_ranking'...","{'name': ['spam', 'lang_mismatch', 'pii', 'not..."
69078,511cfa6a-aca0-45a7-a044-ec69a1a4388d,abe94864-3205-46cd-9a4a-956fd46b89a4,de0a0dde-c714-4e07-a041-d85bc427e6b6,2023-02-04T10:53:24.238619+00:00,"Yes Sir, here is example of checklist for comp...",assistant,en,3,True,False,2.0,False,None,"{'toxicity': 0.00023851396690588444, 'severe_t...",d0bfc03e-9cf9-4465-bf99-53b90e071a8f,ready_for_export,"{'name': ['+1', '_skip_labeling'], 'count': [1...","{'name': ['spam', 'fails_task', 'lang_mismatch..."
69073,baa8199b-ff7a-4d7b-bcc7-9c80d08db3dc,fe34dab0-bab3-4052-8e35-13db58521d9c,de0a0dde-c714-4e07-a041-d85bc427e6b6,2023-02-04T12:23:13.724789+00:00,I am a language model and have no information ...,assistant,en,2,None,True,NaN,False,None,"{'toxicity': 0.0003054749977309257, 'severe_to...",d0bfc03e-9cf9-4465-bf99-53b90e071a

In [27]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 500
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    gradient_checkpointing=True,
)

In [28]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset['train'],
    peft_config=lora_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

/opt/conda/lib/python3.10/site-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


Map:   0%|          | 0/84437 [00:00<?, ? examples/s]

In [31]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [33]:
model.gradient_checkpointing_enable()
trainer.train()

ValueError: PhiForCausalLM does not support gradient checkpointing.